## Создание тома --mount=type=cache  

Вы помните, что каждый слой Docker кешируется отдельно: если COPY или RUN не меняются - слой пересобирать не нужно.  
```dockerfile
   FROM python:3.9-slim
   WORKDIR /app
   COPY requirements.txt .
   RUN pip install -r requirements.txt
   COPY . .
   CMD ["python", "train.py"]
```

Давайте порассуждаем, что произойдёт, если вы добавите новую строку в requirements.txt?  В данном случае все pip-библиотеку будут скачаны заново внутри контейнера.

Поскольку на хост-машине все pip-билиотеки уже скачаны и лежат в папке .cache, то имеет смысл просто пробросить эту папку внутрь контейнера для ускорения. Давайте использовать возможности BuildKit для примонтирования папки кеша.

```dockerfile
   FROM python:3.9-slim
   WORKDIR /app
   COPY requirements.txt .
   RUN --mount=type=cache,id=pip-cache,target=/root/.cache/pip \
       pip install -r requirements.txt
   COPY . .
   CMD ["python", "train.py"]
```
   
Здесь --mount=type=cache создаёт том BuildKit, id позволяет разделять кеши для разных сборок. Если вы попробуете собрать образ дважды подряд и сравните время RUN для особо тяжелых библиотек типа nvidia-cuda/pytorch, то выйгрыш будет колоссальный.


## Предварительная сборка wheel-пакетов

Хотя сборка wheel-пакетов напрямую не относится напрямую к кешированию, обратите внимание, что Wheel-архивы собираются один раз, хранятся в S3 или Artifactory, а в Dockerfile устанавливаются готовые .whl по URL, вместо сборки "с нуля" (поскольку сборка зависимостей может занимать значительное время). Что представляют собой Wheel-архивы? Это просто zip-файл, который нужно распаковать перед установкой. Если в вашей организации есть собственные pip-пакеты, то для ускорения процесса можно вынести сборку зависимостей из образа в систему CI.

Итак, вы заранее собираете все wheel-пакеты, затем ставите только из локальной папки.  

Для создания готовых wheels запустите на хост-машине команду
```python
   pip wheel -r requirements.txt -w wheels/
```

In [ ]:
%%writefile requirements.txt
torch
torchvision

Writing requirements.txt


In [ ]:
!mkdir -p wheels

In [ ]:
%%timeit -n 1 -r 1
!pip wheel -r requirements.txt -w wheels/ -qqq

1min 15s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
%%timeit -n 1 -r 1
!pip install --no-index --find-links=./wheels -r requirements.txt -qqq

8.35 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


Действуя по принципу "тяжело в учении - легко в бою" мы потратили больше минуты на подготовку и меньше 10 секунд на установку огромных библиотек

## Локальный PyPI-прокси или локальное зеркало  

Чтобы вам не обращаться по сети, вы можете проводить все запросы через внутренний прокси devpi одновременно экономя время и трафик.


In [ ]:
%%sh
apt install libffi-dev
pip3 install -U devpi-server devpi-web supervisor -qqq

Reading package lists...
Building dependency tree...
Reading state information...
libffi-dev is already the newest version (3.4.2-4).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
!devpi-server --version

6.17.0


In [ ]:
!devpi-init

2025-09-23 18:51:13,032 INFO  NOCTX Loading node info from /root/.devpi/server/.nodeinfo
2025-09-23 18:51:13,033 INFO  NOCTX generated uuid: 6cc96624e57f44669bb8d788753680c9
2025-09-23 18:51:13,033 INFO  NOCTX wrote nodeinfo to: /root/.devpi/server/.nodeinfo
2025-09-23 18:51:13,034 INFO  NOCTX DB: Creating schema
2025-09-23 18:51:13,231 INFO  [Wtx-1] setting password for user 'root'
2025-09-23 18:51:13,232 INFO  [Wtx-1] created user 'root'
2025-09-23 18:51:13,232 INFO  [Wtx-1] created root user
2025-09-23 18:51:13,232 INFO  [Wtx-1] created root/pypi index
2025-09-23 18:51:13,241 INFO  [Wtx-1] fswriter0: committed at 0


Выше был создан пользователь индекса (root), пароль (по умолчанию пустой) и индекс root/pypi.

Далее генерируем конфиги


In [ ]:
!devpi-gen-config --port 4040 --host 0.0.0.0

It is highly recommended to use a configuration file for devpi-server, see --configfile option.
wrote gen-config/crontab
wrote gen-config/net.devpi.plist
wrote gen-config/launchd-macos.txt
wrote gen-config/nginx-devpi.conf
wrote gen-config/nginx-devpi-caching.conf
wrote gen-config/supervisor-devpi.conf
wrote gen-config/supervisord.conf
wrote gen-config/devpi.service
wrote gen-config/windows-service.txt


Сервер вы запускаете используя основной конфиг с помощью supervisord


In [ ]:
!supervisord -c gen-config/supervisord.conf

In [ ]:
!top

=top - 18:55:31 up 8 min,  0 users,  load average: 1.66, 0.69, 0.33
Tasks:  18 total,   1 running,  16 sleeping,   0 stopped,   1 zombie
%Cpu(s): 48.3 us, 20.7 sy,  0.0 ni, 31.0 id,  0.0 wa,  0.0 hi,  0.0 si,  0.0 st
MiB Mem :  12975.4 total,   8584.8 free,   1053.0 used,   3337.6 buff/cache
MiB Swap:      0.0 total,      0.0 free,      0.0 used.  11631.9 avail Mem 

    PID USER      PR  NI    VIRT    RES    SHR S  %CPU  %MEM     TIME+ COMMAND  
   2052 root      20   0 1753060 346204  21124 S 125.0   2.6   0:58.89 devpi-s+ 
      1 root      20   0     988    512    512 S   0.0   0.0   0:00.03 docker-+ 
      7 root      20   0 1292648  62228  41344 S   0.0   0.5   0:02.58 node     
     15 root      20   0    7376   3456   3200 S   0.0   0.0   0:00.28 oom_mon+ 
     17 root      20   0    7376   1704   1536 S   0.0   0.0   0:00.00 run.sh   
     18 root      20   0 1238020  15172   9344 S   0.0   0.1   0:00.21 kernel_+ 
     30 root      20   0    5808   2048   2048 S   0.0   0.0  

Как вы помните, наш сервер **не** использует https, нам приходится
указывать --trusted-host и далее ip-адрес нашего прокси

In [ ]:
!pip uninstall torch torchvision -y

Found existing installation: torch 2.8.0
Uninstalling torch-2.8.0:
  Successfully uninstalled torch-2.8.0
Found existing installation: torchvision 0.23.0
Uninstalling torchvision-0.23.0:
  Successfully uninstalled torchvision-0.23.0


In [ ]:
%%timeit -n 1 -r 1
!pip install --qqq -i http://localhost:4040/root/pypi/+simple/ -r requirements.txt --trusted-host localhost


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.9/887.9 MB 950.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 85.5 MB/s eta 0:00:00
1min 38s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
%%timeit -n 1 -r 1
!pip install -=qqq -i http://localhost:4040/root/pypi/+simple/ -r requirements.txt --trusted-host localhost

10.8 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


Результат налицо: 11.5 секунд за счёт того, что вы закешировали файлы и теперь нет необходимости ждать пока они скачаются

## Жесткая фиксация зависимостей  

Есть сторонники стабилизировать и не обновлять requirements.txt.



In [ ]:
!pip freeze >requirements.txt
!cat requirements.txt

absl-py==1.4.0
absolufy-imports==0.3.1
accelerate==1.10.1
aiofiles==24.1.0
aiohappyeyeballs==2.6.1
aiohttp==3.12.15
aiosignal==1.4.0
alabaster==1.0.0
albucore==0.0.24
albumentations==2.0.8
ale-py==0.11.2
alembic==1.16.5
altair==5.5.0
annotated-types==0.7.0
antlr4-python3-runtime==4.9.3
anyio==4.10.0
anywidget==0.9.18
argon2-cffi==25.1.0
argon2-cffi-bindings==25.1.0
array_record==0.8.1
arrow==1.3.0
arviz==0.22.0
astropy==7.1.0
astropy-iers-data==0.2025.9.15.0.37.0
astunparse==1.6.3
atpublic==5.1
attrs==25.3.0
audioread==3.0.1
Authlib==1.6.4
autograd==1.8.0
babel==2.17.0
backcall==0.2.0
beartype==0.21.0
beautifulsoup4==4.13.5
betterproto==2.0.0b6
bigframes==2.21.0
bigquery-magics==0.10.3
bleach==6.2.0
blinker==1.9.0
blis==1.3.0
blobfile==3.1.0
blosc2==3.8.0
bokeh==3.7.3
Bottleneck==1.4.2
bqplot==0.12.45
branca==0.8.1
Brotli==1.1.0
build==1.3.0
CacheControl==0.14.3
cachetools==5.5.2
catalogue==2.0.10
certifi==2025.8.3
cffi==2.0.0
Chameleon==4.6.0
chardet==5.2.0
charset-normalizer==3.4.3
c